# Visualizing sector allocations of a mutual fund through time

In [1]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")
source("goofy/misc.common.R")

mfi <- MutualFundsIndia()

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.0     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.1
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Attaching package: ‘xts’

The following objects are masked from ‘package:dplyr’:

    first, last

Loading required package: TTR
Version 0.4-0 included new data defaults. See ?getSymbols.

Attaching package: ‘PerformanceAnalytics’

The following object is masked from ‘package:graphics’:

    legend

Registering fonts with R

Attaching package: ‘gridExtra’

The following object is masked from ‘package:dplyr’:

    combine


Attaching package

In [2]:
#Franklin India Prima Fund-Growth (https://github.com/stockviz/plutons/blob/master/mutual-fund-drawdown.R.ipynb)
scode <- 100473 

sectorAlloc <- mfi$Portfolio() %>%
    filter(SCHEME_CODE == scode & INSTRUMENT == 'EQUITY') %>%
    group_by(PORTFOLIO_DATE, SECTOR) %>%
    summarize(ALLOC = sum(WEIGHTAGE)) %>%
    arrange(PORTFOLIO_DATE, SECTOR) %>%
    collect()

sectorAlloc %>% print(n=100)

Warning message:
“Missing values are always removed in SQL.
Use `SUM(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”

# A tibble: 1,468 x 3
# Groups:   PORTFOLIO_DATE [65]
    PORTFOLIO_DATE SECTOR                    ALLOC
    <date>         <chr>                     <dbl>
  1 2013-10-31     -                         3.69 
  2 2013-10-31     AUTO ANCILLARIES          5.49 
  3 2013-10-31     BANKS                    11.7  
  4 2013-10-31     CEMENT                    3.83 
  5 2013-10-31     CHEMICALS                 5.34 
  6 2013-10-31     CONSTRUCTION PROJECT      2.66 
  7 2013-10-31     CONSUMER DURABLES         2.64 
  8 2013-10-31     CONSUMER NON DURABLES     3.03 
  9 2013-10-31     FERROUS METALS            0.617
 10 2013-10-31     FERTILISERS               2.88 
 11 2013-10-31     FINANCE                   4.97 
 12 2013-10-31     GAS                       2.12 
 13 2013-10-31     INDUSTRIAL CAPITAL GOODS  3.73 
 14 2013-10-31     INDUSTRIAL PRODUCTS      10.4  
 15 2013-10-31     MEDIA AND ENTERTAINMENT   3.44 
 16 2013-10-31     MINERALS/MINING           2.32 
 17 2013-10-31     PESTICIDE

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)